In [1]:
%pylab --no-import-all
%matplotlib inline
from sacla import *

Using matplotlib backend: agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
l=log('2019 SACLA - Shotlog.csv')

In [3]:
import pandas as pd
pd.set_option('display.max_rows', 1000)

In [5]:
l.iloc[65:,:14]

,Run start,last run,Starttime,Endtime,Sample,E_ph,Pre Comment,Pulse Energy,Detector Filter,Beamline filter,Post Comment,Single,Dual,Sx min
,#,#,Time,Time,#/desc,keV,desc,uJ,desc,mm Silicon,desc,Photons,Photons,pulses
65,782314,782316.0,21:56:27,22:04:28,NP 20nm,7.2,dual is not working,465.0,50um Zink,0.1,stopped scan when beam down. Getting the dual ...,3500.0,5000.0,-10700.0
66,782317,782317.0,22:07:23,22:09:28,NP 20nm,7.2,continue the last scan,465.0,50um Zink,0.1,stopped scan because Mn filter in front of dua...,3500.0,5000.0,-10700.0
67,782318,782321.0,22:23:31,22:34:54,"#40, 10.1",7.2,filter in front of dual still,465.0,50um Zink,0.1,NaN,5000.0,2000.0,-10700.0
68,782318,782321.0,22:23:31,22:34:54,"#40, 10.1",7.2,changing attn during run,465.0,50um Zink,0.2,NaN,500.0,250.0,-10700.0
69,782318,782321.0,22:23:31,22:34:54,"#40, 10.1",7.2,"starting from 782321, all Si 0.1mm att",465.0,50um Zink,0.3,NaN,73.0,30.0,-10700.0
70,782318,782321.0,22:23:31,22:34:54,"#40, 10.1",7.2,rotating dual filter,465.0,50um Zink,0.3,didnt work,73.0,30.0,-10700.0
71,782322,782329.0,22:47:27,23:07:31,same as above,7.2,NaN,465.0,50um Zink,0.1,NaN,73.0,30.0,-10700.0
72,782330,782354.0,23:28:54,00:37:14,"50, 20nm high",7.2,important sample,465.0,50um Zink,0.1,NaN,50000.0,5000.0,-10700.0
73,782355,782356.0,00:38:15,00:43:50,"50, 20nm high",7.2,continuing the scan above,465.0,50um Zink,0.1,NaN,50000.0,5000.0,-10700.0


In [6]:
#to run: 97, 82, 83, 84, 87, 88, 93, 94
samples=[
      '10 mid',
      'Fe Foil',
     '51, 10 high',
     '10 agar',
     '10 low',
   '5 mid',
   '20 mid',
#    '5 high',
#    '20nm high',
 
]

In [7]:
#+1:ncpus=1:mem=1gb
#PBS -q psmall
def qsub(commands, jobname='script', start=0, end=0):
    '''
    submits a job (array) using max resources.
    if start==end submit single job, else array from start(inclusive) to end(inclusive)
    use $PBS_ARRAY_INDEX as argument to get index of job in array
    '''
    from subprocess import run, PIPE

    if  isinstance(commands,list):
        commands='\n'.join(str(c) for c in commands)
    pbs = (
f'''
#PBS -V
#PBS -l select=1:ncpus=12:mem=20gb
#PBS -l walltime=24:00:00
#PBS -k oe
#PBS -N {jobname}
'''
        + (f'#PBS -J {start}-{end}' if start != end else f'export PBS_ARRAY_INDEX={start}')
        + 
f'''
{commands}
'''
    )
    #print(pbs)
    p = run(['qsub'], stdout=PIPE, input=pbs, encoding='ascii', cwd='./logs/')
    return (p.returncode, p.stdout)



In [10]:
!mkdir /work/gorkhover/zimmf/np/

In [11]:
def submit(r):
    if 'skip' in r['Run start'][0]: return
    runstart=int((r['Run start'][0]))
    runend=int(r['last run'][0])
    if (runend-runstart)<=2: 
        print('skip',r.name)
        return
    run=str(r.name)
    commands=[
        'hostname',
        'echo $PBS_ARRAY_INDEX',
        'cd /home/gorkhover/zimmf',
        f'python analysis2.py  /UserData/gorkhover/TAIS2019/$PBS_ARRAY_INDEX.h5 --workpath /work/gorkhover/zimmf/ --outpath /work/gorkhover/zimmf/np/ --delete --simple --directrad 75 --allrad --normalize --run {run}'
    ]
    print(qsub(commands,f'{run}-MPInewmask',runstart,runend))
    print(run)
for sample in samples:
    print(sample)
    runs=l.search('Sample',sample)
    isatt=np.array(runs['Beamline filter']!='0')
    runs=runs[isatt]
    runs.apply(submit,axis=1)
    

10 mid
(0, '1957937[].fep02-ib\n')
87
Fe Foil
skip 79
(0, '1957938[].fep02-ib\n')
80
51, 10 high
(0, '1957939[].fep02-ib\n')
82
10 agar
(0, '1957940[].fep02-ib\n')
97
10 low
(0, '1957941[].fep02-ib\n')
93
5 mid
(0, '1957942[].fep02-ib\n')
89
20 mid
(0, '1957943[].fep02-ib\n')
91
